In [1]:
import pickle
import numpy as np
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.data_utils import pad_sequences
import random

In [2]:
positive = pickle.load(open('ml/neuralnet/trainingpositive.p', 'r'))
negative = pickle.load(open('ml/neuralnet/trainingnegative.p', 'r'))

In [3]:
import ml.neuralnet.preProcessFiles as ppf
import ml.nlp.numtext_interconversion as ni
ni.wrapper_normalizer('one hundred and fifty people are gay')

'150 people are gay  '

In [4]:
globalArray = [positive, negative]
training = []
for array in globalArray:
    for subarray in array:
        training.append([ni.wrapper_normalizer(subarray[0].strip()), ni.wrapper_normalizer(subarray[1].strip()), subarray[2]])

In [5]:
for line in training:
    line[0] = ppf.processLine(str(line[0]))
    try:
        line[1] = ppf.processLine(str(line[1]))
    except:
        print line[1]
        z = line[1]
        break


In [6]:
wordlist = []
for array in training:
    temp = array[0].split()
    temp.extend(array[1].split())
    for word in temp:
        if word not in wordlist:
            wordlist.append(word)
    

In [7]:
pickle.dump(wordlist, open('word_list_for_phase_2_CNN.p', 'w'))

In [8]:
finalTraining = []
for array in training:
    finalTraining.append([ppf.tokenize(array[0].split(), wordlist), ppf.tokenize(array[1].split(), wordlist), np.array(array[2])])

In [9]:
finalTraining[0][0]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 9,
 10,
 6,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 6,
 18,
 19,
 20,
 7,
 9,
 9,
 21,
 22]

In [10]:
pickle.dump(finalTraining, open('phase2_training_set.p', 'w'))

In [11]:
maxlen = 15

In [12]:
random.shuffle(finalTraining)
X_train, X_test, y_train, y_test = [], [], [], []
for i in range(int(0.9*len(finalTraining))):
    X_train.append(np.array([pad_sequences([finalTraining[i][0]], maxlen),pad_sequences([finalTraining[i][1]], maxlen)]))
    y_train.append(np.array(finalTraining[i][2]))
    
for i in range(int(0.9*len(finalTraining)), len(finalTraining)):
    X_test.append(np.array([pad_sequences([finalTraining[i][0]], maxlen),pad_sequences([finalTraining[i][1]], maxlen)]))
    y_test.append(np.array(finalTraining[i][2]))
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [13]:
X_train = X_train.reshape([-1, 2, maxlen, 1])
X_test = X_test.reshape([-1, 2, maxlen, 1])
X_train.shape

(2145, 2, 15, 1)

In [14]:
convnet = None
convnet = input_data(shape=[None, 2, maxlen, 1], name='input')

convnet = conv_2d(convnet, 8, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 16, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 32, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 32, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', n_classes=2, learning_rate=0.01, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet)

In [15]:
model.fit({'input':X_train}, {'targets':y_train}, n_epoch=50)

Training Step: 1699  | total loss: 0.51100 | time: 0.135s
| Adam | epoch: 050 | loss: 0.51100 -- iter: 2112/2145
Training Step: 1700  | total loss: 0.51533 | time: 0.139s
| Adam | epoch: 050 | loss: 0.51533 -- iter: 2145/2145
--


In [16]:
arr = np.array([pad_sequences([ppf.tokenize(ppf.processLine("2 out of 8 people eat meat").split(), wordlist)], maxlen), pad_sequences([ppf.tokenize(ppf.processLine("few people eat meat").split(), wordlist)], maxlen)])

In [17]:
arr = arr.reshape([-1, 2, maxlen, 1])

In [18]:
model.predict(arr)

array([[ 0.90629268,  0.09370727]], dtype=float32)